In [24]:
!nvidia-smi

Thu Jun 10 22:20:21 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8    14W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'nvidia-kernel-common-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-325-updates' for glob 'nvidia*'
Note, selecting 'nvidia-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-driver-binary' for glob 'nvidia*'
Note, selecting 'nvidia-331-dev' for glob 'nvidia*'
Note, selecting 'nvidia-304-updates-dev' for glob 'nvidia*'
Note, selecting 'nvidia-compute-utils-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-384-dev' for glob 'nvidia*'
Note, selecting 'nvidia-libopencl1-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-driver-440-server' for glob 'nvidia*'
Note, selecting 'nvidia-340-updates-uvm' for glob 'nvidia*'
Note, selecting 'nvidia-dkms-450-server' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-common' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-source-440-server' for glob 'nvidia*'
Note, selecting 'nvidia-331-updates-uvm' for glob 'nvidi

In [ ]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2

--2021-06-10 02:49:43--  https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Resolving developer.nvidia.com (developer.nvidia.com)... 152.199.0.24
Connecting to developer.nvidia.com (developer.nvidia.com)|152.199.0.24|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://developer.nvidia.com/compute/cuda/9.2/prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 [following]
--2021-06-10 02:49:44--  https://developer.nvidia.com/compute/cuda/9.2/prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Reusing existing connection to developer.nvidia.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/9.2/secure/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb?GyOEM9BxsvACnbuCoOO10hebpRY6HiG_uB0gmund5jpmA88cJKoY_qO_OovI8teJ6PNRcREVbBIzDleit6rLJciJD36pc_yR8hFAUoC3N3O2-R4xX

In [5]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0


In [6]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-c4hrntoy
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-c4hrntoy
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp37-none-any.whl size=4305 sha256=2d4b1ed1302466a912e174ff53c21e84855fbe695fcab4d3df07479704c021a4
  Stored in directory: /tmp/pip-ephem-wheel-cache-f4t4uoeg/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


In [7]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [ ]:
%%cu
#include <stdio.h>
#include <stdlib.h>
__global__ void add(int *a, int *b, int *c) {
*c = *a + *b;
}
int main() {
int a, b, c;
// host copies of variables a, b & c
int *d_a, *d_b, *d_c;
// device copies of variables a, b & c
int size = sizeof(int);
// Allocate space for device copies of a, b, c
cudaMalloc((void **)&d_a, size);
cudaMalloc((void **)&d_b, size);
cudaMalloc((void **)&d_c, size);
// Setup input values  
c = 0;
a = 3;
b = 5;
// Copy inputs to device
cudaMemcpy(d_a, &a, size, cudaMemcpyHostToDevice);
  cudaMemcpy(d_b, &b, size, cudaMemcpyHostToDevice);
// Launch add() kernel on GPU
add<<<1,1>>>(d_a, d_b, d_c);
// Copy result back to host
cudaError err = cudaMemcpy(&c, d_c, size, cudaMemcpyDeviceToHost);
  if(err!=cudaSuccess) {
      printf("CUDA error copying to Host: %s\n", cudaGetErrorString(err));
  }
printf("result is %d\n",c);
// Cleanup
cudaFree(d_a);
cudaFree(d_b);
cudaFree(d_c);
return 0;
}

result is 8



In [ ]:
%%cu
#include <stdio.h>
__global__ void hello() {
    printf("Hello world from device");
}
int main() {
    hello<<<1, 1>>>();
    printf("Hello world from host");
    return 0;
}

Hello world from host


In [ ]:
%%cu
#include <stdio.h>

// Size of array
#define N 1000000

// Kernel
__global__ void add_vectors(double *a, double *b, double *c)
{
	int id = blockDim.x * blockIdx.x + threadIdx.x;
	if(id < N) c[id] = a[id] + b[id];
}

// Main program
int main()
{
	// Number of bytes to allocate for N doubles
	size_t bytes = N*sizeof(double);

	// Allocate memory for arrays A, B, and C on host
	double *A = (double*)malloc(bytes);
	double *B = (double*)malloc(bytes);
	double *C = (double*)malloc(bytes);

	// Allocate memory for arrays d_A, d_B, and d_C on device
	double *d_A, *d_B, *d_C;
	cudaMalloc(&d_A, bytes);
	cudaMalloc(&d_B, bytes);
	cudaMalloc(&d_C, bytes);

	// Fill host arrays A and B
	for(int i=0; i<N; i++)
	{
		A[i] = 1.0;
		B[i] = 2.0;
	}

	// Copy data from host arrays A and B to device arrays d_A and d_B
	cudaMemcpy(d_A, A, bytes, cudaMemcpyHostToDevice);
	cudaMemcpy(d_B, B, bytes, cudaMemcpyHostToDevice);

	// Set execution configuration parameters
	//		thr_per_blk: number of CUDA threads per grid block
	//		blk_in_grid: number of blocks in grid
	int thr_per_blk = 256;
	int blk_in_grid = ceil( float(N) / thr_per_blk );

	// Launch kernel
	add_vectors<<< blk_in_grid, thr_per_blk >>>(d_A, d_B, d_C);

	// Copy data from device array d_C to host array C
	cudaMemcpy(C, d_C, bytes, cudaMemcpyDeviceToHost);

	// Verify results	
	for(int i=0; i<N; i++)
	{
		if(C[i] != 3.0)
		{ 
			printf("\nError: value of C[%d] = %d instead of 3.0\n\n", i, C[i]);
			exit(-1);
		}
	}	

	// Free CPU memory
	free(A);
	free(B);
	free(C);

	// Free GPU memory
	cudaFree(d_A);
	cudaFree(d_B);
	cudaFree(d_C);

	printf("\n---------------------------\n");
	printf("__SUCCESS__\n");
	printf("---------------------------\n");
	printf("N                 = %d\n", N);
	printf("Threads Per Block = %d\n", thr_per_blk);
	printf("Blocks In Grid    = %d\n", blk_in_grid);
	printf("---------------------------\n\n");

	return 0;
}


---------------------------
__SUCCESS__
---------------------------
N                 = 1000000
Threads Per Block = 256
Blocks In Grid    = 3907
---------------------------




In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [23]:
%%cu
#include <iostream>

#define STB_IMAGE_IMPLEMENTATION
#include "/content/drive/My Drive/Colab Notebooks/LibreriaSTBIMAGE/stb_image.h"

#define STB_IMAGE_WRITE_IMPLEMENTATION
#include "/content/drive/My Drive/Colab Notebooks/LibreriaSTBIMAGE/stb_image_write.h"

using namespace std;

__constant__ int BLUR_SIZE = 50;

__global__
void convertToGrayScaleKernel(unsigned char* Pbw, const unsigned char* Pin, int width, int height)
{
  int col = blockIdx.x * blockDim.x + threadIdx.x;
  int row = blockIdx.y * blockDim.y + threadIdx.y;

  if(col < width && row < height) {
    int grey_loc = row*width + col;
    int rgb_loc = 3*grey_loc;

    unsigned char r = Pin[rgb_loc  ];
    unsigned char g = Pin[rgb_loc+1];
    unsigned char b = Pin[rgb_loc+2];
    Pbw[grey_loc] = 0.21f*r + 0.71f*g + 0.07f*b;
  }
}

/**
 * Blur the BW image (Kernel)
 */
__global__
void blurKernel(unsigned char* Pout, const unsigned char* Pbw, int width, int height)
{
  int col = blockIdx.x * blockDim.x + threadIdx.x;
  int row = blockIdx.y * blockDim.y + threadIdx.y;

  if(col < width && row < height) {
    int loc = row*width + col;

    int pixval = 0;
    int pixels = 0;
    for(int i = -BLUR_SIZE; i < BLUR_SIZE+1; ++i) {
      for(int j = -BLUR_SIZE; j < BLUR_SIZE+1; ++j) {
        int blurRow = row + i;
        int blurCol = col + j;
        if(blurRow > -1 && blurRow < height && blurCol > -1 && blurCol < width) {
          pixval += Pbw[blurRow*width + blurCol];
          pixels++;
        }
      }
    }
    Pout[loc] = (unsigned char)(pixval/pixels);
  }
}
int main(int argc, char* argv[])
{
    int width, height, channels;
    unsigned char* imageIn = stbi_load("/content/drive/My Drive/Colab Notebooks/LibreriaSTBIMAGE/image1.jpg", &width, &height, &channels, 0);
    cout << "Imported image " << "image1.jpg" << " (" << width << " x " << height << ") with " << channels << " channels" << endl;
    int size = width*height;
    unsigned char* out_image1 = new unsigned char[size];
    unsigned char* out_image2 = new unsigned char[size];
 
    unsigned char* d_Pin;
    cudaMalloc((void**)&d_Pin, size*3);
    cudaMemcpy((void*)d_Pin, (void*)imageIn, size*3, cudaMemcpyHostToDevice);
 
    unsigned char* d_Pin2;
    cudaMalloc((void**)&d_Pin2, size*3);
    cudaMemcpy((void*)d_Pin2, (void*)imageIn, size*3, cudaMemcpyHostToDevice);
 
    unsigned char* d_Pin3;
    cudaMalloc((void**)&d_Pin3, size);

    unsigned char* grayscaleout;
    unsigned char* blurout;
    cudaMalloc((void**)&grayscaleout, size);
    cudaMalloc((void**)&blurout, size);

    dim3 dimGrid(ceil(width/16.0f),ceil(height/16.0f),1);
    dim3 dimBlock(16,16,1);
    cout << "Launching a (" << dimGrid.x << " x " << dimGrid.y << " x " << dimGrid.z << ") grid." << endl;
    cout << "Total number of threads: " << dimGrid.x*dimGrid.y*dimGrid.z*16*16 << endl;
    cout << "Number of pixels: " << width*height << endl;
 
    convertToGrayScaleKernel<<<dimGrid,dimBlock>>>(grayscaleout, d_Pin, width, height);
    blurKernel<<<dimGrid,dimBlock>>>(blurout, d_Pin2, width, height);

    cudaMemcpy((void*)out_image1, (void*)grayscaleout, size, cudaMemcpyDeviceToHost);
    cudaMemcpy((void*)out_image2, (void*)blurout, size, cudaMemcpyDeviceToHost);
 
    stbi_write_bmp("imageGrayScale_out.bmp", width, height, 1, (void*)out_image1);
    stbi_write_bmp("imageBlur_out.bmp", width, height, 1, (void*)out_image2);

    stbi_image_free(imageIn);
    delete [] out_image1;
    delete [] out_image2;
    cudaFree(d_Pin);
    cudaFree(d_Pin2);
    cudaFree(d_Pin3);
    //cudaFree(d_Pbw);
    //cudaFree(d_Pout);
 }

Imported image image3.png (1200 x 796) with 3 channels
Launching a (75 x 50 x 1) grid.
Total number of threads: 960000
Number of pixels: 955200

